In [1]:
import numpy as np

from pyoints import (
    storage,
    Extent,
    transformation,
    filters,
    registration,
    normals,
)

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from copy import copy

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

%matplotlib inline

# Подготовка 

In [ ]:
A = storage.loadPly('./data/ArmadilloBack/ArmadilloBack_0.ply')
B = storage.loadPly('./data/ArmadilloBack/ArmadilloBack_30.ply')
C = storage.loadPly('./data/ArmadilloBack/ArmadilloBack_60.ply')

print('A:', A.shape, A.dtype.descr, '\nB:', B.shape, B.dtype.descr, '\nC:', C.shape, C.dtype.descr)

: 

In [ ]:
r = 0.3
A = A[list(filters.ball(A.indexKD(), r))]
B = B[list(filters.ball(B.indexKD(), r))]
C = C[list(filters.ball(C.indexKD(), r))]

print('A:', A.shape, A.dtype.descr, '\nB:', B.shape, B.dtype.descr, '\nC:', C.shape, C.dtype.descr)

: 

In [ ]:
def plot(**clouds):
    coord_df = pd.DataFrame()
    
    for key in clouds:
        coords_dict = {
            'x': clouds[key].coords.T[0,].tolist(),
            'y': clouds[key].coords.T[1,].tolist(),
            'z': clouds[key].coords.T[2,].tolist()
        }
        
        coords_dict['cloud'] = [key for _ in range(len(coords_dict['x']))]
        
        coord_df = pd.concat((coord_df, pd.DataFrame(coords_dict)), ignore_index=True)
        
    fig = px.scatter_3d(coord_df, x='x', y='y', z='z', color='cloud')
    
    fig.update_layout(
        width=1200,
        height=1000
    )
    
    fig.update_traces(marker_size = 5)

    fig.show()

: 

In [ ]:
plot(A=A, B=B, C=C)

: 

## Трансформации

In [ ]:
transforms = {
    'best': ([0, 0, 0], [-40*np.pi/180, -180*np.pi/180, 0], [0, -30*np.pi/180, -10*np.pi/180]),
    'v1': ([0, 0, 0], [-35*np.pi/180, -185*np.pi/180, 0], [0, -33*np.pi/180, 0]),
    'v2': ([0, 0, 0], [-44*np.pi/180, -177*np.pi/180, 0], [0, -31*np.pi/180, 3*np.pi/180]),
    'v3': ([0, 0, 0], [-39*np.pi/180, -182*np.pi/180, 2*np.pi/180], [0, -28*np.pi/180, 0]),
}

: 

In [ ]:
A_2 = copy(A)
B_2 = copy(B)
C_2 = copy(C)

transform = transforms['v1']
T_A = transformation.r_matrix(transform[0])
A_2.transform(T_A)
T_B = transformation.r_matrix(transform[1])
B_2.transform(T_B)
T_C = transformation.r_matrix(transform[2])
C_2.transform(T_C);

: 

In [ ]:
plot(axes_lims=axes_lims, A=A_2, B=B_2, C=C_2)

: 

# ICP алгоритм

In [ ]:
def plot(title='', **clouds):
    coord_df = pd.DataFrame()
    for key in clouds:
        coords_dict = {
            'x': clouds[key].T[0,].tolist(),
            'y': clouds[key].T[1,].tolist(),
            'z': clouds[key].T[2,].tolist()
        }
        coords_dict['cloud'] = [key for _ in range(len(coords_dict['x']))]
        coord_df = pd.concat((coord_df, pd.DataFrame(coords_dict)), ignore_index=True)
        
    fig = px.scatter_3d(coord_df, x='x', y='y', z='z', color='cloud')
    fig.update_layout(width=1200, height=1000, title=title)
    
    fig.update_traces(marker_size = 5)

    fig.show()
    
    
COLORS =['#3366CC', '#DC3912', '#FF9900', '#109618', '#990099', '#0099C6', '#DD4477', '#66AA00', '#B82E2E','#316395'] 
   
   
def get_rmse(results: dict, title='RMSE'):
    results = pd.DataFrame(results)
        
    fig = go.Figure()

    labels = [str(num) for num in range(1, len(results)+1)]
    columns = results.columns.to_list()

    bar_colors = COLORS[:len(columns)]
    bar_colors = iter(bar_colors)

    for column in columns:
        values = [round(count, 3) for count in results[column]]
        print(f'Final RMSE {column}:', values[-1])
        fig.add_trace(go.Bar(x=labels, y=values, name=column, marker_color=next(bar_colors), text=values))
        fig.update_layout(height=1000,width=2000, title=title, barmode='group')
    fig.show()

: 

In [ ]:
%%time
transforms = {
    'best': ([0, 0, 0], [-40*np.pi/180, -180*np.pi/180, 0], [0, -30*np.pi/180, -10*np.pi/180]),
    'v1': ([0, 0, 0], [-35*np.pi/180, -185*np.pi/180, 0], [0, -33*np.pi/180, 0]),
    'v2': ([0, 0, 0], [-44*np.pi/180, -177*np.pi/180, 0], [0, -31*np.pi/180, 3*np.pi/180]),
    'v3': ([0, 0, 0], [-39*np.pi/180, -182*np.pi/180, 2*np.pi/180], [0, -28*np.pi/180, 0]),
}


d_th = 2
radii = [d_th, d_th, d_th]
icp = registration.ICP(
    radii,
    max_iter=50,
    max_change_ratio=0.00001,
    k=1
)

reports = {}

for transform_key in transforms:
    A_2 = copy(A)
    B_2 = copy(B)
    C_2 = copy(C)

    transform = transforms[transform_key]
    T_A = transformation.r_matrix(transform[0])
    A_2.transform(T_A)
    T_B = transformation.r_matrix(transform[1])
    B_2.transform(T_B)
    T_C = transformation.r_matrix(transform[2])
    C_2.transform(T_C)
    
    coords_dict = {
        'A': A_2.coords,
        'B': B_2.coords,
        'C': C_2.coords
    }
    
    T_dict, pairs_dict, report = icp(coords_dict)
    
    reports[transform_key] = report['RMSE']
    
    for key in coords_dict:
        coords_dict[key] = transformation.transform(coords_dict[key], T_dict[key])
    
    
    title = f'Transform {transform_key}'
    plot(title=title, **coords_dict)
    
get_rmse(reports)
    

: 

# NICP алгоритм

In [ ]:
coords_dict = {
    'A': A.coords,
    'B': B.coords,
    'C': C.coords
}

normals_dict = {
    key: normals.fit_normals(coords_dict[key], k=5, preferred=[0, -1, 0])
    for key in coords_dict
}


fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection='3d')
ax.set_xlim(axes_lims[0], axes_lims[3])
ax.set_ylim(axes_lims[1], axes_lims[4])
ax.set_zlim(axes_lims[2], axes_lims[5])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.scatter(*A.coords.T, c=normals_dict['A'][:, 2], cmap='coolwarm')
for coord, normal in zip(coords_dict['A'], normals_dict['A']):
    ax.plot(*np.vstack([coord, coord + normal*0.3]).T, color='black')
plt.show()


: 

In [ ]:
%%time
transforms = {
    'best': ([0, 0, 0], [-40*np.pi/180, -180*np.pi/180, 0], [0, -30*np.pi/180, -10*np.pi/180]),
    'v1': ([0, 0, 0], [-35*np.pi/180, -185*np.pi/180, 0], [0, -33*np.pi/180, 0]),
    'v2': ([0, 0, 0], [-44*np.pi/180, -177*np.pi/180, 0], [0, -31*np.pi/180, 3*np.pi/180]),
    'v3': ([0, 0, 0], [-39*np.pi/180, -182*np.pi/180, 2*np.pi/180], [0, -28*np.pi/180, 0]),
}


d_th = 2
n_th = np.sin(15 * np.pi / 180)
radii = [d_th, d_th, d_th, n_th, n_th, n_th]
nicp = registration.ICP(
    radii,
    max_iter=50,
    max_change_ratio=0.000001,
    update_normals=True,
    k=1
)

reports = {}

for transform_key in transforms:
    A_2 = copy(A)
    B_2 = copy(B)
    C_2 = copy(C)

    transform = transforms[transform_key]
    T_A = transformation.r_matrix(transform[0])
    A_2.transform(T_A)
    T_B = transformation.r_matrix(transform[1])
    B_2.transform(T_B)
    T_C = transformation.r_matrix(transform[2])
    C_2.transform(T_C)
    
    coords_dict = {
        'A': A_2.coords,
        'B': B_2.coords,
        'C': C_2.coords
    }
    
    normals_dict = {
        key: normals.fit_normals(coords_dict[key], k=5, preferred=[0, -1, 0])
        for key in coords_dict
    }
    
    T_dict, pairs_dict, report = nicp(coords_dict, normals_dict)
    
    reports[transform_key] = report['RMSE']
    
    for key in coords_dict:
        coords_dict[key] = transformation.transform(coords_dict[key], T_dict[key])
    
    
    title = f'Transform {transform_key}'
    plot(title=title, **coords_dict)
    
get_rmse(reports)
    

: 

# Анимация

In [ ]:
A_2 = copy(A)
B_2 = copy(B)
C_2 = copy(C)

transform = transforms['v1']
T_A = transformation.r_matrix(transform[0])
A_2.transform(T_A)
T_B = transformation.r_matrix(transform[1])
B_2.transform(T_B)
T_C = transformation.r_matrix(transform[2])
C_2.transform(T_C)

coords_dict = {
    'A': A_2.coords,
    'B': B_2.coords,
    'C': C_2.coords
}

normals_dict = {
    key: normals.fit_normals(coords_dict[key], k=5, preferred=[0, -1, 0])
    for key in coords_dict
}

T_dict, pairs_dict, report = nicp(coords_dict, normals_dict)


axes_lims = Extent([
    A_2.extent().center - 0.5 * A_2.extent().ranges.max(),
    A_2.extent().center + 0.5 * A_2.extent().ranges.max()
])
colors = {'A': 'green', 'B': 'blue', 'C': 'red'}

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection='3d')
ax.set_xlim(axes_lims[0], axes_lims[3])
ax.set_ylim(axes_lims[1], axes_lims[4])
ax.set_zlim(axes_lims[2], axes_lims[5])
fig.tight_layout()

# initializing plot
artists={
    key: ax.plot([],[],[], '.', color=colors[key], label=key)[0]
    for key in coords_dict
}
ax.legend()

# collecting the roto-translation matrices
T_iter = [{key: np.eye(4) for key in coords_dict}] + report['T']

def animate(i):
    # updates the frame
    ax.set_xlabel('Iteration %i' % i)
    for key in coords_dict:
            coords = transformation.transform(coords_dict[key], T_iter[i][key])
            artists[key].set_data(coords[:, 0], coords[:, 1])
            artists[key].set_3d_properties(coords[:, 2])
    return artists.values()

# creates the animation
anim = animation.FuncAnimation(fig, animate, frames=range(len(T_iter)), interval=250, blit=True)

# save as GIF
anim.save('./data/nicp.gif', writer='pillow', fps=10)
plt.close()
# display as HTML (online version only)
HTML(anim.to_jshtml())

: 

: 